In [1]:
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
from scipy import special

from Source import RM

Using TensorFlow backend.


In [2]:
def add_masking(received_code, msg_size):
    masking = np.ones((msg_size,2))
    if np.var(received_code) > 1.9:
        received_code = np.hstack((masking, received_code))
    elif np.var(received_code) > 1.45:
        masking[:,1] = -1*masking[:,1]
        received_code = np.hstack((masking, received_code))
    elif np.var(received_code) > 1.225:
        masking[:,0] = -1*masking[:,0]
        received_code = np.hstack((masking, received_code))
    else:
        masking = -1 * masking
        received_code = np.hstack((masking, received_code))
    return received_code

In [3]:
m = 4
r = 1
length = 2**m

G = RM.G_matrix(length, m, r)
G_p = RM.G_prime(G, m)

if r == 1:
    masking_length=0
else:
    masking_length=int(np.math.factorial(m)/(np.math.factorial(2)*np.math.factorial(m-2)))

In [4]:
model = load_model('Model/model_(1,4)_p6_softmax.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [5]:
file_buffer = 'Result/model_(1,4)_softmax.csv'
model_f = open(file_buffer, 'w')
file_buffer = 'Result/rm.csv'
rm_f = open(file_buffer, 'w')

In [6]:
max_error = 500
for i in range(0,11):
    model_BER = 0
    rm_BER = 0
    total_error_count = 0
    total_model_count = 0
    code_num = 0
    while(1):
        msg = np.random.randint(2, size = (1,m+r))
        send_code, received_code = RM.Encoding(msg, G, i, length)
        
            
        model_ae_code = model.predict(received_code, verbose=0)
        
        result = np.argmax(model_ae_code, axis=1)
        model_msg = np.zeros((1,5))
        model_msg[0] = list(format(result[0], 'b').zfill(5))

        rm_estimated_code = RM.Decoding(received_code, m, length, r, G, G_p)
        
        model_count = ((msg + model_msg)%2).sum()
        error_count = ((msg + rm_estimated_code)%2).sum()
        
#         model_BER += model_count / (msg.shape[0] * msg.shape[1])
#         rm_BER += error_count / (msg.shape[0] * msg.shape[1])
        
        total_model_count += model_count
        total_error_count += error_count
        
        code_num+=1
        
        if total_model_count >= max_error and total_error_count >= max_error:
            model_BER = total_model_count/(5*code_num)
            rm_BER = total_error_count/(5*code_num)
            break
        if code_num >= 10**5:
            model_BER = total_model_count/(5*code_num)
            rm_BER = total_error_count/(5*code_num)
            break
        
    write_buffer = 'Eb_No, %f, ' %(i)
    model_f.write(write_buffer)
    rm_f.write(write_buffer)
    
    write_buffer = 'BER, %.10f\n' % (model_BER)
    model_f.write(write_buffer)
    write_buffer = 'BER, %.10f\n' % (rm_BER)
    rm_f.write(write_buffer)
    print(code_num)
    print('model', model_BER)
    print('rm', rm_BER)

model_f.close()
rm_f.close()

1211
model 0.0916597853014038
rm 0.0829066886870355
2077
model 0.05642753972075108
rm 0.04814636494944632
4527
model 0.025668212944554892
rm 0.02217804285398719
10907
model 0.012359035481800678
rm 0.0091867607958192
38598
model 0.0035027721643608478
rm 0.0025959894295041193
100000
model 0.000574
rm 0.000456
100000
model 0.000124
rm 5.8e-05


KeyboardInterrupt: 

In [ ]:
print(model_msg)

In [ ]:
print(msg)